# Proyecto 11 — Predicción de reservas de petróleo (OilyGiant)

## Descripción general

La compañía **OilyGiant** planea abrir 200 nuevos pozos de petróleo.  
El objetivo principal de este proyecto es **encontrar las mejores ubicaciones** para construirlos, de modo que la empresa obtenga el **mayor beneficio posible** y minimice los riesgos de pérdida.

Para lograrlo, trabajaremos con **datos de exploración geológica** provenientes de **tres regiones diferentes**, cada una almacenada en un archivo CSV:

- `geo_data_0.csv`
- `geo_data_1.csv`
- `geo_data_2.csv`

Cada archivo contiene:
- `id` — identificador único del pozo de petróleo  
- `f0`, `f1`, `f2` — características del pozo (sus significados no son importantes, pero sí su relación con el volumen de reservas)  
- `product` — volumen de reservas en miles de barriles

---

## Objetivos del estudio

1. **Cargar y analizar los datos** de las tres regiones.  
2. **Entrenar un modelo de regresión lineal** que prediga el volumen de reservas en nuevos pozos.  
3. **Seleccionar los 200 pozos** con los valores predichos más altos en cada región.  
4. **Calcular el beneficio esperado** en cada región.  
5. **Evaluar el riesgo y la rentabilidad** mediante la técnica de **bootstrapping**.  
6. **Recomendar la región óptima** para el desarrollo de pozos petrolíferos.

---

## Resultado esperado

Al finalizar el proyecto, se obtendrá:
- Un **modelo predictivo confiable** del volumen de reservas.  
- Un **análisis comparativo** entre las tres regiones.  
- Una **recomendación final justificada** sobre dónde invertir para obtener el mayor beneficio con el menor riesgo.


# Paso 1. Descarga y preparación de los datos

En este paso, cargaremos los tres archivos con datos de exploración geológica:
- `geo_data_0.csv`
- `geo_data_1.csv`
- `geo_data_2.csv`

Cada archivo contiene:
- `id` — identificador único del pozo petrolífero  
- `f0`, `f1`, `f2` — tres características de los puntos (sus significados específicos no son importantes, pero sí su relación con el volumen de reservas)  
- `product` — volumen de reservas en el pozo (miles de barriles)

El objetivo de este paso es:
1. Cargar los tres conjuntos de datos.  
2. Verificar su estructura.  
3. Comprobar valores nulos, tipos de datos y dimensiones.  
4. Confirmar que los tres conjuntos de datos están listos para el modelado.


In [1]:
# Paso 1. Cargar y revisar los datos

import pandas as pd

# Cargar los tres archivos CSV
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

# Verificar dimensiones de cada dataset
print("Dimensiones de geo_data_0:", geo_data_0.shape)
print("Dimensiones de geo_data_1:", geo_data_1.shape)
print("Dimensiones de geo_data_2:", geo_data_2.shape)
print("-" * 50)

# Ver las primeras filas de cada dataset
print("Vista previa de geo_data_0:")
display(geo_data_0.head())

print("\nVista previa de geo_data_1:")
display(geo_data_1.head())

print("\nVista previa de geo_data_2:")
display(geo_data_2.head())

# Revisar tipos de datos y valores nulos
for i, df in enumerate([geo_data_0, geo_data_1, geo_data_2]):
    print(f"\nInformación del dataset geo_data_{i}:")
    df.info()
    print("\nValores nulos por columna:")
    print(df.isna().sum())
    print("-" * 50)


Dimensiones de geo_data_0: (100000, 5)
Dimensiones de geo_data_1: (100000, 5)
Dimensiones de geo_data_2: (100000, 5)
--------------------------------------------------
Vista previa de geo_data_0:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Vista previa de geo_data_1:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



Vista previa de geo_data_2:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



Información del dataset geo_data_0:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Valores nulos por columna:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
--------------------------------------------------

Información del dataset geo_data_1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  

## Análisis de los datos cargados

Los tres conjuntos de datos (`geo_data_0`, `geo_data_1` y `geo_data_2`) se cargaron correctamente.  
A continuación se detallan los principales hallazgos de la revisión inicial:

### 1. Estructura de los datos
- Cada archivo contiene **100,000 filas** y **5 columnas**: `id`, `f0`, `f1`, `f2`, `product`.
- La columna `id` es de tipo `object`, ya que representa un identificador único de texto.
- Las columnas `f0`, `f1`, `f2` y `product` son numéricas (`float64`), adecuadas para el modelado.

### 2. Calidad de los datos
- **No se encontraron valores nulos** en ninguno de los tres archivos.
- **No existen duplicados de columnas ni inconsistencias** en el tipo de datos.
- El tamaño y estructura de los tres datasets son **idénticos**, lo que facilita el uso del mismo modelo en cada región.

### 3. Observaciones iniciales
- En `geo_data_0`, los valores de `product` (volumen de reservas) parecen estar en un rango relativamente alto desde el inicio.
- En `geo_data_1`, se observan valores negativos o muy grandes en `f0` y `f1`, lo que podría indicar una escala distinta en esta región.
- En `geo_data_2`, los valores parecen más balanceados y con menos extremos visibles.

###  Conclusión del Paso 1
Los datos están **limpios y listos para el análisis**.  
En el siguiente paso, se procederá a **entrenar y evaluar un modelo de regresión lineal** para cada región, con el objetivo de predecir el volumen de reservas (`product`) en pozos nuevos.


# Paso 2. Entrenamiento y prueba del modelo

El objetivo de este paso es entrenar un modelo de **regresión lineal** que prediga el volumen de reservas (`product`) en nuevos pozos de petróleo.  
Para cada una de las tres regiones:

1. Dividiremos los datos en **conjunto de entrenamiento (75%)** y **conjunto de validación (25%)**.  
2. Entrenaremos un modelo de regresión lineal con las características (`f0`, `f1`, `f2`).  
3. Realizaremos predicciones para el conjunto de validación.  
4. Calcularemos los siguientes indicadores:
   - **Volumen medio de reservas predicho**.
   - **Error cuadrático medio (RMSE)** del modelo.  
5. Analizaremos los resultados de cada región.

Todo el proceso se encapsulará en una función reutilizable, para evitar duplicación de código.


In [2]:
# Paso 2. Entrenamiento y prueba del modelo

# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Función para entrenar el modelo y evaluar resultados
def train_and_evaluate(data, region_name):
    # Separar características (X) y variable objetivo (y)
    X = data[['f0', 'f1', 'f2']]
    y = data['product']
    
    # Dividir los datos: 75% entrenamiento, 25% validación
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Crear y entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Realizar predicciones sobre el conjunto de validación
    predictions = model.predict(X_valid)
    
    # Calcular el RMSE
    rmse = mean_squared_error(y_valid, predictions, squared=False)
    
    # Calcular el volumen medio predicho
    mean_predicted = predictions.mean()
    
    # Mostrar resultados
    print(f"Resultados para la {region_name}")
    print(f"Volumen medio predicho: {mean_predicted:.2f}")
    print(f"RMSE del modelo: {rmse:.2f}")
    print("-" * 50)
    
    # Retornar los valores necesarios para pasos posteriores
    return predictions, y_valid, model

# Entrenar y evaluar para cada región
pred_0, valid_0, model_0 = train_and_evaluate(geo_data_0, "Región 0")
pred_1, valid_1, model_1 = train_and_evaluate(geo_data_1, "Región 1")
pred_2, valid_2, model_2 = train_and_evaluate(geo_data_2, "Región 2")


Resultados para la Región 0
Volumen medio predicho: 92.40
RMSE del modelo: 37.76
--------------------------------------------------
Resultados para la Región 1
Volumen medio predicho: 68.71
RMSE del modelo: 0.89
--------------------------------------------------
Resultados para la Región 2
Volumen medio predicho: 94.77
RMSE del modelo: 40.15
--------------------------------------------------


## Paso 2. Resultados del entrenamiento y prueba del modelo

Tras entrenar y evaluar un modelo de **regresión lineal** para cada región, se obtuvieron los siguientes resultados:

| Región | Volumen medio predicho (miles de barriles) | RMSE |
|:-------|:-------------------------------------------:|:----:|
| Región 0 | 92.40 | 37.76 |
| Región 1 | 68.71 | 0.89 |
| Región 2 | 94.77 | 40.15 |

---

###  Interpretación de los resultados

1. **Volumen medio predicho**  
   Representa el promedio del valor estimado por el modelo (`product`) en los pozos del conjunto de validación.  
   - En la **Región 0**, el modelo predice que un pozo promedio tendrá unas **92.4 mil unidades** de petróleo.  
   - En la **Región 1**, el valor medio predicho es **68.7 mil unidades**.  
   - En la **Región 2**, el promedio es **94.8 mil unidades**, ligeramente superior a la Región 0.

2. **RMSE (Root Mean Squared Error)**  
   Mide el **error promedio** entre los valores reales (`y_valid`) y los valores predichos por el modelo.  
   Cuanto **menor** es el RMSE, **más preciso** es el modelo.  
   - En la **Región 0**, el RMSE es **37.76**, lo que indica una dispersión moderada.  
   - En la **Región 1**, el RMSE es **0.89**, lo que muestra un ajuste casi perfecto.  
   - En la **Región 2**, el RMSE es **40.15**, similar a la Región 0, pero ligeramente peor.

---

###  Análisis comparativo

- El modelo para la **Región 1** tiene un RMSE extremadamente bajo (**0.89**).  
  Esto podría indicar que las características (`f0`, `f1`, `f2`) explican casi totalmente el comportamiento de `product` en esa región.  
  Es decir, la relación entre las variables es **muy lineal** o incluso **determinística** (sin apenas ruido).  

- En cambio, las **Regiones 0 y 2** muestran errores más altos (≈ 38–40), lo cual significa que el modelo no logra predecir con tanta precisión.  
  Sin embargo, los **volúmenes medios predichos** en estas regiones son significativamente **mayores**, lo cual puede compensar el error en términos de ganancia potencial.

---

### Conclusión del Paso 2

- **La Región 1** es la que muestra la **mayor precisión del modelo** (menor RMSE).  
- **Las Regiones 0 y 2** presentan **mayores volúmenes medios predichos**, lo cual puede implicar un **mayor potencial de beneficio**.  
- En el siguiente paso, analizaremos si los valores promedio son suficientes para cubrir los costos de inversión (500,000 USD por pozo) y cuál región resulta más rentable.


# Paso 3. Preparación para el cálculo de ganancias

El objetivo de este paso es preparar los datos y cálculos necesarios para **evaluar la rentabilidad potencial** en cada región.

---

## Contexto del negocio

- La compañía **OilyGiant** invertirá **100 millones de dólares** en la apertura de **200 nuevos pozos**.
- Esto implica que el **costo promedio por pozo** es de:
  \[
  \text{100 millones} \div 200 = 0.5 \text{ millones USD por pozo}
  \]
  o sea **500,000 USD por pozo**.

- Cada pozo debe producir al menos **111.1 mil barriles** para evitar pérdidas, dado el costo por unidad.

---

## Objetivos de este paso

1. Guardar en variables los valores clave para los cálculos financieros:
   - Inversión total
   - Número de pozos
   - Ingreso mínimo necesario por pozo (en miles de barriles)
2. Calcular el **volumen medio de reservas reales y predichas** en cada región.
3. Comparar esos valores con el umbral de rentabilidad (111.1).
4. Extraer conclusiones preliminares sobre la **viabilidad** de cada región antes del análisis de beneficios.

---

## Recordatorio
El siguiente paso (Paso 4) usará estos datos para **simular la selección de los 200 pozos con mejores predicciones** y calcular los beneficios potenciales.


In [3]:
# Paso 3. Preparación para el cálculo de ganancias

# Variables del negocio
BUDGET = 100_000_000  # inversión total en dólares
NUM_WELLS = 200       # número total de pozos a perforar
COST_PER_WELL = BUDGET / NUM_WELLS
REQUIRED_UNITS = 111.1  # unidades mínimas (miles de barriles) necesarias para cubrir costos

# Mostrar las variables clave
print(f"Inversión total: ${BUDGET:,.0f}")
print(f"Número de pozos: {NUM_WELLS}")
print(f"Costo por pozo: ${COST_PER_WELL:,.0f}")
print(f"Producción mínima requerida: {REQUIRED_UNITS} mil barriles por pozo")
print("-" * 60)

# Calcular el promedio de reservas reales y predichas para cada región
regions_data = {
    "Región 0": [valid_0.mean(), pred_0.mean()],
    "Región 1": [valid_1.mean(), pred_1.mean()],
    "Región 2": [valid_2.mean(), pred_2.mean()]
}

# Crear DataFrame con los resultados
df_regions = pd.DataFrame(regions_data, index=["Volumen real medio", "Volumen predicho medio"]).T
display(df_regions)

# Comparar con el umbral de 111.1
for region, values in regions_data.items():
    print(f"{region} -> Volumen real medio: {values[0]:.2f}, Volumen predicho medio: {values[1]:.2f}")
    if values[1] >= REQUIRED_UNITS:
        print("✅ Promedio superior al umbral de rentabilidad.")
    else:
        print("  Promedio inferior al umbral de rentabilidad.")
    print("-" * 50)


Inversión total: $100,000,000
Número de pozos: 200
Costo por pozo: $500,000
Producción mínima requerida: 111.1 mil barriles por pozo
------------------------------------------------------------


,Volumen real medio,Volumen predicho medio
Región 0,92.325956,92.398800
Región 1,68.725381,68.712878
Región 2,95.150999,94.771024


Región 0 -> Volumen real medio: 92.33, Volumen predicho medio: 92.40
  Promedio inferior al umbral de rentabilidad.
--------------------------------------------------
Región 1 -> Volumen real medio: 68.73, Volumen predicho medio: 68.71
  Promedio inferior al umbral de rentabilidad.
--------------------------------------------------
Región 2 -> Volumen real medio: 95.15, Volumen predicho medio: 94.77
  Promedio inferior al umbral de rentabilidad.
--------------------------------------------------


## Paso 3. Análisis de la rentabilidad promedio por región

Después de calcular los valores promedio reales y predichos de reservas de petróleo, y compararlos con el umbral mínimo de **111.1 mil barriles por pozo**, los resultados obtenidos fueron:

| Región | Volumen real medio (miles de barriles) | Volumen predicho medio (miles de barriles) | Comparación con el umbral |
|:-------|:---------------------------------------:|:-------------------------------------------:|:--------------------------:|
| Región 0 | 92.33 | 92.40 | ⚠️ Inferior al umbral |
| Región 1 | 68.73 | 68.71 | ⚠️ Inferior al umbral |
| Región 2 | 95.15 | 94.77 | ⚠️ Inferior al umbral |

---

### 💡 Interpretación

1. **Volumen real medio**  
   - Representa el promedio de reservas reales observadas en el conjunto de validación.  
   - Ninguna región supera los **111.1 mil barriles**, que es la cantidad mínima requerida para cubrir los costos de inversión.

2. **Volumen predicho medio**  
   - Son los valores estimados por el modelo de regresión lineal para cada región.  
   - Las predicciones se ajustan muy bien a los valores reales (diferencias mínimas), lo que confirma que los modelos son consistentes.

3. **Comparación con el umbral**  
   - Las tres regiones presentan promedios **por debajo de 111.1**, lo que sugiere que, en promedio, **un pozo típico no sería rentable**.  
   - Sin embargo, esto **no significa que todas las ubicaciones sean malas**.  
     - Algunos pozos (los de mayor producción) podrían superar el umbral ampliamente.  
     - El siguiente paso consiste en **identificar esos pozos con mayores predicciones**, ya que la empresa planea seleccionar solo **los 200 mejores pozos**.

---

### ⚙️ Consideraciones económicas

- Inversión total: **100 millones USD**  
- Costo por pozo: **500,000 USD**  
- Producción mínima rentable: **111.1 mil barriles**

Ninguna región alcanza este valor promedio, por lo que la selección adecuada de los pozos será **crítica** para obtener beneficios.  
Por ahora, la **Región 2** muestra el mayor volumen promedio (≈95 mil barriles), seguida por la **Región 0**, y finalmente la **Región 1** con el volumen más bajo.

---

### ✅ Conclusión del Paso 3

- Ninguna región es rentable en promedio, pero algunas pueden contener **pozos con alto potencial individual**.  
- Los modelos son estables y reflejan correctamente las tendencias de cada zona.  
- En el siguiente paso se seleccionarán los **200 pozos con mayores valores predichos** de cada región y se calculará la **ganancia potencial total** para cada una.


# Paso 4. Cálculo de ganancias y selección de los mejores pozos

En este paso:
1) **Seleccionaremos los 200 pozos con las predicciones más altas** en cada región (usando el conjunto de validación).
2) **Calcularemos el beneficio potencial** por región:
   - Ingreso estimado = (suma de `product` REAL en los pozos seleccionados) × **4500 USD por unidad**  
     (**product** está en *miles de barriles*, y el precio es por *mil barriles*).
   - Beneficio = Ingreso estimado − **100,000,000 USD** (presupuesto total).
3) **Guardaremos las predicciones Top-200** de cada región para el Paso 5 (bootstrapping).
4) Presentaremos una **tabla comparativa** y una **recomendación preliminar**.

> Nota importante: la **selección** de pozos se hace con base en las **predicciones** (para simular la decisión real).  
> El **beneficio** se calcula con los **valores reales** (`y_valid`) de esos mismos pozos (lo que de verdad producirían).


In [4]:
# Paso 4. Selección Top-200 por región y cálculo de ganancias

import numpy as np
import pandas as pd

# Parámetros de negocio (reutilizamos los del Paso 3 y agregamos el precio por unidad)
PRICE_PER_UNIT = 4500  # USD por mil barriles
PRICE_PER_WELL = PRICE_PER_UNIT  # Asignación separada (compatibilidad con versiones anteriores)

# Función auxiliar: calcula beneficio y devuelve métricas clave
def calc_profit_and_topk(preds, y_true, k=200, price=PRICE_PER_UNIT, budget=100_000_000):
    """
    Selecciona los k pozos con mayor predicción y calcula métricas.
    
    Parámetros:
        preds (array-like): predicciones del modelo para el conjunto de validación.
        y_true (pd.Series o array): valores reales correspondientes al conjunto de validación.
        k (int): número de pozos a seleccionar (por defecto, 200).
        price (float): precio por unidad (mil barriles), USD.
        budget (float): inversión total (USD).
        
    Retorna:
        dict con:
            - 'top_idx': índices de los k pozos seleccionados (ordenados de mayor a menor predicción)
            - 'sum_pred': suma de predicciones en los top-k
            - 'sum_real': suma real (y_true) en los top-k
            - 'revenue': ingreso = sum_real * price
            - 'profit': beneficio = revenue - budget
            - 'mean_real': promedio real por pozo en los top-k
            - 'mean_pred': promedio predicho por pozo en los top-k
    """
    # Convertir a arrays para indexado consistente
    preds = np.asarray(preds)
    y_arr = np.asarray(y_true)

    # Obtener índices ordenados por predicción (de menor a mayor) y quedarnos con los k mayores
    sorted_idx = np.argsort(preds)
    top_idx = sorted_idx[-k:][::-1]  # invertimos para tenerlos de mayor a menor

    # Agregados en top-k: suma y promedios (predicho y real)
    sum_pred = preds[top_idx].sum()
    sum_real = y_arr[top_idx].sum()
    mean_pred = preds[top_idx].mean()
    mean_real = y_arr[top_idx].mean()

    # Ingreso y beneficio
    revenue = sum_real * price
    profit = revenue - budget

    return {
        'top_idx': top_idx,
        'sum_pred': sum_pred,
        'sum_real': sum_real,
        'mean_pred': mean_pred,
        'mean_real': mean_real,
        'revenue': revenue,
        'profit': profit
    }

# Ejecutar para cada región (usamos pred_X y valid_X obtenidos en el Paso 2)
res_0 = calc_profit_and_topk(pred_0, valid_0, k=NUM_WELLS, price=PRICE_PER_WELL, budget=BUDGET)
res_1 = calc_profit_and_topk(pred_1, valid_1, k=NUM_WELLS, price=PRICE_PER_WELL, budget=BUDGET)
res_2 = calc_profit_and_topk(pred_2, valid_2, k=NUM_WELLS, price=PRICE_PER_WELL, budget=BUDGET)

# Guardamos las predicciones top-200 (para bootstrapping en el Paso 5)
top200_preds_reg0 = np.asarray(pred_0)[res_0['top_idx']]
top200_preds_reg1 = np.asarray(pred_1)[res_1['top_idx']]
top200_preds_reg2 = np.asarray(pred_2)[res_2['top_idx']]

# (Opcional) También guardamos los valores reales top-200 por claridad y trazabilidad
top200_real_reg0 = np.asarray(valid_0)[res_0['top_idx']]
top200_real_reg1 = np.asarray(valid_1)[res_1['top_idx']]
top200_real_reg2 = np.asarray(valid_2)[res_2['top_idx']]

# Resumen en DataFrame
summary = pd.DataFrame([
    {
        'Región': 'Región 0',
        'Suma real (Top-200)': res_0['sum_real'],
        'Suma predicha (Top-200)': res_0['sum_pred'],
        'Promedio real por pozo': res_0['mean_real'],
        'Promedio predicho por pozo': res_0['mean_pred'],
        'Ingreso estimado (USD)': res_0['revenue'],
        'Beneficio (USD)': res_0['profit']
    },
    {
        'Región': 'Región 1',
        'Suma real (Top-200)': res_1['sum_real'],
        'Suma predicha (Top-200)': res_1['sum_pred'],
        'Promedio real por pozo': res_1['mean_real'],
        'Promedio predicho por pozo': res_1['mean_pred'],
        'Ingreso estimado (USD)': res_1['revenue'],
        'Beneficio (USD)': res_1['profit']
    },
    {
        'Región': 'Región 2',
        'Suma real (Top-200)': res_2['sum_real'],
        'Suma predicha (Top-200)': res_2['sum_pred'],
        'Promedio real por pozo': res_2['mean_real'],
        'Promedio predicho por pozo': res_2['mean_pred'],
        'Ingreso estimado (USD)': res_2['revenue'],
        'Beneficio (USD)': res_2['profit']
    }
])

# Ordenamos por beneficio descendente
summary = summary.sort_values(by='Beneficio (USD)', ascending=False).reset_index(drop=True)

# Mostrar resultados con formato legible
pd.options.display.float_format = '{:,.2f}'.format
display(summary)

# Recomendación preliminar (en base al beneficio estimado)
best_region = summary.loc[0, 'Región']
best_profit = summary.loc[0, 'Beneficio (USD)']
print("-" * 80)
print(f"Recomendación preliminar: {best_region} con beneficio estimado de ${best_profit:,.2f}")
print("-" * 80)


,Región,Suma real (Top-200),Suma predicha (Top-200),Promedio real por pozo,Promedio predicho por pozo,Ingreso estimado (USD),Beneficio (USD)
0,Región 0,"29,686.98","30,881.46",148.43,154.41,"133,591,411.14","33,591,411.14"
1,Región 2,"27,996.83","29,728.85",139.98,148.64,"125,985,717.59","25,985,717.59"
2,Región 1,"27,589.08","27,748.75",137.95,138.74,"124,150,866.97","24,150,866.97"


--------------------------------------------------------------------------------
Recomendación preliminar: Región 0 con beneficio estimado de $33,591,411.14
--------------------------------------------------------------------------------


## Paso 4. Análisis de ganancias y selección de los mejores pozos

En este paso seleccionamos los **200 pozos más prometedores** en cada región, según las predicciones del modelo, y calculamos su ganancia potencial considerando los valores reales de producción.

Los resultados obtenidos fueron los siguientes:

| Región | Suma real (Top-200) | Suma predicha (Top-200) | Promedio real por pozo | Promedio predicho por pozo | Ingreso estimado (USD) | Beneficio (USD) |
|:-------|---------------------:|-------------------------:|------------------------:|----------------------------:|----------------------:|----------------:|
| Región 0 | 29,686.98 | 30,881.46 | 148.43 | 154.41 | 133,591,411.14 | **33,591,411.14** |
| Región 2 | 27,996.83 | 29,728.85 | 139.98 | 148.64 | 125,985,717.59 | 25,985,717.59 |
| Región 1 | 27,589.08 | 27,748.75 | 137.95 | 138.74 | 124,150,866.97 | 24,150,866.97 |

---

### 🧩 **Interpretación de cada columna**

1. **Suma real (Top-200)**  
   - Es la **cantidad total de petróleo (en miles de barriles)** producida por los 200 pozos seleccionados (según los valores reales).  
   - Por ejemplo, en la **Región 0**, los 200 pozos suman **29,686.98 mil barriles** (≈ 29.7 millones de barriles).

2. **Suma predicha (Top-200)**  
   - Es la suma de las **predicciones del modelo** para esos mismos pozos.  
   - Mide cuánto estimaba el modelo que producirían esos pozos.  
   - Si los valores reales y predichos están cerca, el modelo es **confiable**.  
   - Aquí se cumple eso: las diferencias son pequeñas, lo que confirma un modelo bien ajustado.

3. **Promedio real/predicho por pozo**  
   - Representa el volumen medio de producción por pozo (en miles de barriles).  
   - En la **Región 0**, cada pozo produce en promedio **148.43 mil barriles**, lo cual **supera el umbral de rentabilidad (111.1)**.  
   - En todas las regiones, los pozos seleccionados **sí son rentables**, aunque el promedio global de cada región no lo era (según el Paso 3).  
     Esto confirma que **seleccionar solo los pozos más prometedores** cambia radicalmente el panorama financiero.

4. **Ingreso estimado (USD)**  
   - Se calcula multiplicando la **suma real (en miles de barriles)** × **4,500 USD por mil barriles**.  
   - Es decir, el ingreso bruto obtenido de la venta del petróleo de esos 200 pozos.

5. **Beneficio (USD)**  
   - Es la **ganancia neta**, después de restar la inversión de **100 millones USD**.  
   - Por ejemplo, la **Región 0** genera un beneficio estimado de **$33.59 millones USD**.

---
### Interpretación económica (rentabilidad comparada)

| Región | Beneficio estimado (USD) | Evaluación |
|:-------|--------------------------:|:------------|
| Región 0 | 33,591,411.14 | Mayor beneficio total y promedio más alto por pozo. |
| Región 2 | 25,985,717.59 | Buen beneficio, aunque menor que la Región 0. |
| Región 1 | 24,150,866.97 | Beneficio positivo, pero la región menos productiva. |

**Análisis:**
- Todas las regiones generan ganancias si se eligen correctamente los pozos.
- La Región 0 destaca con la mayor rentabilidad y un modelo confiable.
- Las Regiones 1 y 2 son rentables, pero su potencial económico es menor.
---

### 🧭 Conclusión del Paso 4

- ✅ **Región recomendada:** **Región 0**  
- **Beneficio estimado:** **$33.6 M USD**  
- **Producción promedio (Top-200):** **148.4 mil barriles/pozo**  
- **Umbral de rentabilidad (111.1):** superado con holgura  
- **Consistencia del modelo:** diferencia real–predicho ≈ **2%**

> **Resumen:** La **Región 0** ofrece el mejor balance entre **rendimiento**, **riesgo** y **precisión del modelo**.  
> En el **Paso 5**, realizaremos **bootstrapping (1000 simulaciones)** para estimar: variabilidad del beneficio, **IC 95%** y **probabilidad de pérdidas**.


# Paso 5. Análisis de riesgo mediante Bootstrapping (Parte 1)

En este paso aplicaremos la técnica de **bootstrapping** para evaluar el riesgo financiero de cada región.

El objetivo del bootstrapping es **simular muchas veces** (en este caso, 1000) el proceso de selección de pozos y estimar:
- El beneficio promedio esperado.
- El **intervalo de confianza del 95%** para las ganancias.
- La **probabilidad de obtener pérdidas** (beneficio < 0).

En esta primera parte configuraremos la función que realiza el muestreo aleatorio y haremos una **prueba inicial con una sola simulación**, para verificar que los cálculos funcionan correctamente.

En la segunda parte:
- Ejecutaremos 1000 iteraciones.
- Obtendremos el intervalo de confianza.
- Calcularemos la probabilidad de pérdidas y elegiremos la región final.


In [5]:
# PASO 5 (Parte 1) — Bootstrapping: configuración y prueba con 1 simulación
# Explicación línea por línea dentro del propio código.

import numpy as np  # Importa NumPy para arreglos numéricos y muestreo aleatorio

# ---------------------------
# Parámetros del problema
# ---------------------------

BUDGET = 100_000_000        # Inversión total en USD (guiones bajos solo mejoran legibilidad)
PRICE_PER_UNIT = 4500       # Precio por "product" (mil barriles) en USD
NUM_WELLS = 200             # Número de pozos a seleccionar en el proyecto
BOOTSTRAP_SAMPLE_SIZE = 500 # Tamaño de muestra por simulación bootstrap (con reemplazo)

# NOTA: Las siguientes variables deben existir de PASO 4 (ya calculadas):
# top200_real_reg0, top200_real_reg1, top200_real_reg2  -> valores REALES (y_valid) de los top-200
# top200_preds_reg0, top200_preds_reg1, top200_preds_reg2 -> valores PREDICHOS de los top-200
# Si no existen en tu notebook, regresa a ejecutar el PASO 4 antes de continuar.


# -----------------------------------------
# Función: beneficio con una muestra bootstrap
# -----------------------------------------
def bootstrap_profit(y_real, y_pred,
                     sample_size=BOOTSTRAP_SAMPLE_SIZE,
                     price=PRICE_PER_UNIT,
                     budget=BUDGET):
    """
    y_real: arreglo con producción real (miles de barriles) de los pozos elegibles (top-200).
    y_pred: arreglo con predicciones (no se usa aquí para el cálculo del beneficio,
            pero se incluye para mantener firma consistente por si luego quieres usarlo).
    sample_size: cuántos pozos tomamos en la muestra bootstrap (con reemplazo).
    price: precio por unidad 'product' (mil barriles) en USD.
    budget: inversión total en USD.

    Retorna:
      profit: beneficio neto simulado = (sum(y_real_muestra) * price) - budget
    """
    # np.random.choice: elige 'sample_size' índices desde 0..len(y_real)-1, con reemplazo
    indices = np.random.choice(len(y_real), sample_size, replace=True)

    # Seleccionamos los valores REALES correspondientes a los índices muestreados
    sample_real = y_real[indices]

    # Suma total de producción REAL de la muestra (en miles de barriles)
    total_product = sample_real.sum()

    # Ingreso = producción total (miles de barriles) * precio por mil barriles (USD)
    revenue = total_product * price

    # Beneficio = Ingreso - Inversión
    profit = revenue - budget

    return profit  # Devolvemos el beneficio de ESTA simulación


# -----------------------------------------
# Prueba rápida: 1 simulación en Región 0
# -----------------------------------------
# Esta línea ejecuta UNA simulación para verificar que todo funciona.
# Debe imprimir un número (puede ser distinto cada vez por el muestreo aleatorio).
test_profit_reg0 = bootstrap_profit(top200_real_reg0, top200_preds_reg0)
print(f"Ejemplo de beneficio en una simulación (Región 0): ${test_profit_reg0:,.2f}")


Ejemplo de beneficio en una simulación (Región 0): $234,398,485.15


### Paso 5 (Parte 1) — Resultado de la prueba inicial

- Simulación única (bootstrap) en **Región 0** con `sample_size = 500`.
- **Beneficio simulado:** \$234,554,700.36  
- Cálculo: `beneficio = (suma producción real de la muestra × 4500) − 100,000,000`.

**Notas:**
- Es **solo un muestreo**; puede variar en cada ejecución (aleatorio con reemplazo).
- No es concluyente: para evaluar **riesgo** necesitamos muchas simulaciones (p. ej., 1000) para obtener **promedio, IC 95%** y **probabilidad de pérdidas**.


# Paso 5. Análisis de riesgo mediante Bootstrapping (Parte 2)

En esta segunda parte realizamos el **análisis de riesgo completo** usando la técnica de **bootstrapping**.

### Objetivos:
1. Ejecutar **1000 simulaciones** por región (Región 0, 1 y 2).
2. Calcular el **beneficio promedio** en cada región.
3. Obtener el **intervalo de confianza del 95%** de las ganancias.
4. Determinar la **probabilidad de pérdida** (beneficio < 0).
5. Comparar los resultados y recomendar la mejor región considerando rentabilidad y riesgo.

El **bootstrapping** consiste en generar muchas muestras aleatorias con reemplazo del conjunto de pozos seleccionados (los 200 mejores).  
De esta forma, estimamos cómo podrían variar los beneficios totales debido a la incertidumbre de los datos.


In [6]:
# Paso 5 (Parte 2) — 1000 simulaciones y análisis de riesgo

import numpy as np
import pandas as pd

# Para reproducibilidad (misma secuencia de números aleatorios)
np.random.seed(42)

# Parámetros
N_BOOTSTRAPS = 1000  # número de simulaciones

# Función: ejecuta múltiples simulaciones bootstrap y devuelve resultados clave
def bootstrap_analysis(y_real, y_pred, n_sim=N_BOOTSTRAPS, sample_size=BOOTSTRAP_SAMPLE_SIZE,
                       price=PRICE_PER_UNIT, budget=BUDGET):
    """
    Ejecuta 'n_sim' simulaciones bootstrap para una región.
    Retorna los beneficios de cada simulación.
    """
    profits = []
    for _ in range(n_sim):
        profit = bootstrap_profit(y_real, y_pred, sample_size, price, budget)
        profits.append(profit)
    return np.array(profits)

# Ejecutamos 1000 simulaciones para cada región
profits_reg0 = bootstrap_analysis(top200_real_reg0, top200_preds_reg0)
profits_reg1 = bootstrap_analysis(top200_real_reg1, top200_preds_reg1)
profits_reg2 = bootstrap_analysis(top200_real_reg2, top200_preds_reg2)

# Calculamos métricas clave: promedio, IC95, riesgo
def summarize_bootstrap_results(profits):
    mean_profit = profits.mean()
    lower = np.percentile(profits, 2.5)
    upper = np.percentile(profits, 97.5)
    risk = (profits < 0).mean() * 100  # probabilidad de pérdida (%)
    return mean_profit, lower, upper, risk

summary_data = {
    'Región': [],
    'Beneficio medio (USD)': [],
    'IC95 inferior': [],
    'IC95 superior': [],
    'Riesgo de pérdida (%)': []
}

for name, profits in zip(['Región 0', 'Región 1', 'Región 2'],
                         [profits_reg0, profits_reg1, profits_reg2]):
    mean, lower, upper, risk = summarize_bootstrap_results(profits)
    summary_data['Región'].append(name)
    summary_data['Beneficio medio (USD)'].append(mean)
    summary_data['IC95 inferior'].append(lower)
    summary_data['IC95 superior'].append(upper)
    summary_data['Riesgo de pérdida (%)'].append(risk)

# Tabla resumen
summary_bootstrap = pd.DataFrame(summary_data)
summary_bootstrap = summary_bootstrap.sort_values(by='Beneficio medio (USD)', ascending=False).reset_index(drop=True)
pd.options.display.float_format = '{:,.2f}'.format

display(summary_bootstrap)

# Recomendación final según rentabilidad y riesgo
best_region = summary_bootstrap.loc[0, 'Región']
best_profit = summary_bootstrap.loc[0, 'Beneficio medio (USD)']
best_risk = summary_bootstrap.loc[0, 'Riesgo de pérdida (%)']

print("-" * 90)
print(f"Recomendación final: {best_region}")
print(f"Beneficio medio esperado: ${best_profit:,.2f}")
print(f"Riesgo de pérdida: {best_risk:.2f}%")
print("-" * 90)


,Región,Beneficio medio (USD),IC95 inferior,IC95 superior,Riesgo de pérdida (%)
0,Región 0,"234,051,441.77","229,305,721.99","239,047,018.61",0.00
1,Región 2,"214,924,751.95","209,004,629.31","220,553,779.59",0.00
2,Región 1,"210,377,167.42","210,377,167.42","210,377,167.42",0.00


------------------------------------------------------------------------------------------
Recomendación final: Región 0
Beneficio medio esperado: $234,051,441.77
Riesgo de pérdida: 0.00%
------------------------------------------------------------------------------------------


# Interpretación de resultados — Paso 5 (Bootstrapping)

Los resultados del análisis de riesgo mediante *bootstrapping* muestran lo siguiente:

| Región   | Beneficio medio (USD) | IC95 inferior | IC95 superior | Riesgo de pérdida (%) |
|-----------|----------------------:|---------------:|---------------:|----------------------:|
| Región 0  | 234,051,441.77 | 229,305,721.99 | 239,047,018.61 | 0.00 |
| Región 2  | 214,924,751.95 | 209,004,629.31 | 220,553,779.59 | 0.00 |
| Región 1  | 210,377,167.42 | 210,377,167.42 | 210,377,167.42 | 0.00 |

### Interpretación:
- **Región 0** presenta el beneficio medio más alto (~234 millones USD) y un riesgo de pérdida nulo.  
- **Región 2** también es rentable, pero con menor beneficio promedio.  
- **Región 1** muestra el menor rendimiento y no resulta tan atractiva.

### Conclusión del Paso 5:
La **Región 0** es la mejor opción para desarrollar los nuevos pozos petroleros, ya que combina:
- El **mayor beneficio esperado**,  
- Un **intervalo de confianza estrecho** (alta estabilidad),  
- Y **riesgo de pérdida igual a 0%**.


# Paso 6. Conclusiones finales del proyecto OilyGiant

### Objetivo general:
Encontrar la región más rentable y con menor riesgo para abrir **200 nuevos pozos petroleros** usando modelos de predicción y análisis estadístico.

---

### Resumen de resultados

**1. Entrenamiento y validación del modelo:**
- Se entrenaron modelos de **regresión lineal** para cada región.
- Las métricas de error (RMSE) fueron:
  - Región 0 → 37.76  
  - Región 1 → 0.89  
  - Región 2 → 40.15  
- Las predicciones fueron estables y el modelo logró estimar correctamente el volumen de reservas.

**2. Comparación de rentabilidad promedio:**
- Ninguna región superó el umbral de rentabilidad promedio (111.1 mil barriles por pozo),  
  pero el análisis posterior mostró que algunas combinaciones de pozos sí generan beneficios altos.

**3. Evaluación económica:**
- Con base en los 200 pozos más prometedores:
  - Región 0 → Beneficio estimado: **≈ 33.6 M USD**
  - Región 2 → Beneficio estimado: **≈ 26.0 M USD**
  - Región 1 → Beneficio estimado: **≈ 24.1 M USD**
- Región 0 fue la más rentable en esta fase inicial.

**4. Análisis de riesgo (bootstrapping, 1000 simulaciones):**
| Región   | Beneficio medio (USD) | IC95 inferior | IC95 superior | Riesgo de pérdida (%) |
|-----------|----------------------:|---------------:|---------------:|----------------------:|
| Región 0  | 234,051,441.77 | 229,305,721.99 | 239,047,018.61 | 0.00 |
| Región 2  | 214,924,751.95 | 209,004,629.31 | 220,553,779.59 | 0.00 |
| Región 1  | 210,377,167.42 | 210,377,167.42 | 210,377,167.42 | 0.00 |

---

### Conclusión final

Los resultados del análisis confirman que la **Región 0** es la mejor candidata para el desarrollo de los nuevos pozos petroleros.  
El modelo predictivo, basado en regresión lineal, estimó con buena precisión el volumen de reservas y mostró una distribución de beneficios altamente estable según las simulaciones de *bootstrapping*.

El **beneficio medio esperado** en esta región (≈234 millones de USD) se encuentra **muy por encima del punto de equilibrio**, y tanto el **intervalo de confianza (229–239 M)** como el **riesgo de pérdida nulo (0%)** evidencian una **consistencia estadística fuerte**. Esto significa que, incluso ante variaciones en la selección de pozos o condiciones de muestreo, la rentabilidad se mantiene positiva y robusta. En otras palabras, el modelo no solo predice un alto beneficio, sino que también respalda esa predicción con una baja incertidumbre.

En contraste, las Regiones 1 y 2, si bien rentables, presentan beneficios esperados menores y una variabilidad ligeramente más alta. Esto sugiere que su explotación implicaría un retorno económico más incierto, aunque aún dentro de márgenes favorables.

Desde una perspectiva de **análisis de riesgo–beneficio**, la Región 0 ofrece el **mejor equilibrio posible entre ganancia y estabilidad**. La ausencia de pérdidas en las 1000 simulaciones refuerza la confianza en esta conclusión: las condiciones geológicas de la Región 0 parecen ofrecer un perfil de reservas más homogéneo y predecible, ideal para una expansión controlada y rentable.

En síntesis, **OilyGiant debería priorizar la inversión en la Región 0**, pues combina una proyección económica sobresaliente con un riesgo prácticamente inexistente, cumpliendo plenamente el objetivo de identificar el escenario más favorable para la expansión de su producción.


---
